Using Spotify API and spotipy python library

In [ ]:
pip install spotipy 

In [ ]:
import sys
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import itertools
import pandas as pd
import datetime as datetime

SPOTIPY_CLIENT_ID=''
SPOTIPY_CLIENT_SECRET=''

In [ ]:
client_credentials_manager = SpotifyClientCredentials(SPOTIPY_CLIENT_ID,SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=20)

In [ ]:
playlists = sp.user_playlists('spotify')
playlist_ids = []
while playlists:
    for i, playlist in enumerate(playlists['items']):
        playlist_ids.append(playlist['id'])
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

In [ ]:
trackslist = [] 
for j,i in enumerate(playlist_ids):
    if j%50 == 0:
        print(j)
    tracks = sp.playlist_tracks(i, limit = 100)
    trackslist.append(tracks['items'])
    while tracks['next']:
        tracks = sp.next(tracks)
        trackslist.append(tracks['items'])

In [ ]:
int(trackslist[0][0]['track']['album']['release_date'][0:4])
int(trackslist[0][0]['track']['album']['release_date'][0:4])
decade = 1900
newlist = []
templist = []
for y in range(decade,2030,10):
  for i in range(len(trackslist)): 
    for j in range(len(trackslist[i])-1):
      try:
        currentYear = int(trackslist[i][j]['track']['album']['release_date'][0:4])
        if( currentYear >= y and currentYear < (y+10)):
          templist.append([trackslist[i][j]])
      except:
        continue
  if(templist != []):
    print(y)
    newlist.append(templist)
    templist = []

  #newlist = [[[{1}],{2},3,4,5],[],[],[],[],[],[]]

1900
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020


Take artist from tracks

In [ ]:
X=30
tracks = [j['track'] for i in newlist[10] for j in i]
artistids = [j['id'] for i in tracks if i!=None for j in i['artists']]
artistids = pd.Series(artistids).drop_duplicates()
artistids = artistids.dropna()
artists = sp.artists(artistids[:X])['artists']
for i in range(X,len(artistids),X):
    artists.append(sp.artists(artistids[i:(i+X)])['artists'])
artists = artists[:X] + [j for i in artists[X:] for j in i]

In [ ]:
for j in range(len(artists)):
        artists[j]['followers'] = artists[j]['followers']['total']
artists = pd.DataFrame.from_dict(artists)
artists = artists.drop_duplicates(subset = ['uri'])
len(artists)

14610

In [ ]:
artists  = artists[[ 'followers', 'genres','id','name','popularity','uri']]
allart = artists.append(artists)
allart = allart.drop_duplicates(subset = ['id'])


allart.columns = ['followers', 'genres','spotify id','Label','popularity','uri']
strgnrs = ['_'.join(i) for i in allart['genres']]
allart['strgnrs'] = strgnrs

In [ ]:
#allart.to_csv('NEWNEW-artists>>>DECADE-2020<<<_nodes_gephi.csv', index = False)

Create Matrix

In [ ]:
sum = 0
gnrs = [i for i in allart['genres'] if i!= []]
c = [list(itertools.combinations(i,2)) for i in gnrs ]
a = list(itertools.chain.from_iterable((i, i[::-1]) for c_ in c for i in c_))
dfa = pd.DataFrame(a)
dfgenrs = pd.pivot_table(dfa, index=0, columns=1, aggfunc='size', fill_value=0)
print(a[0])

('modern rock', 'pop')


In [ ]:
stam = [col for col, val in dfgenrs.sum().iteritems() if type(val)==int and int(val) < 5]

In [ ]:
dfgenrs.drop(stam,axis=1,inplace=True) #drop col
dfgenrs.drop(stam,inplace=True) #drop row 


In [ ]:
dfgenrs.shape

(1909, 1909)

In [ ]:
dfa.to_csv('NEW-network_nodes_final2010.csv',index = False)
dfgenrs.to_csv('New-network_matrix_final2010.csv', index = False)

----------------------------------------------------------------------

get Node and edges props



In [ ]:
#Reading in artists data and removing artists without any genres tagged
artists = allart
artists = artists.loc[artists['genres'] != '[]']
len(artists)

14610

In [ ]:
gnrs = []
for i in artists.index:
    tmpdict = {'genre': artists['strgnrs'][i].split("_"),'popularity': artists['popularity'][i],
              'followers': artists['followers'][i]}
    gnrs.append(tmpdict)
c = [{'genres': list(itertools.combinations(i['genre'],2)), 'popularity': i['popularity'], 'followers': i['followers']} for i in gnrs]

In [ ]:
#Creating a list of edges from the combinations created above
edges = [{'edge': j, 'population': i['popularity'],'followers': i['followers']}  for i in c for j in i['genres']]
edges = pd.DataFrame.from_dict(edges)
edges = pd.DataFrame(edges.groupby(['edge']).agg(['mean','count']))
edges.columns = ['mean popularity','count','mean followers', 'all count']
edges = edges.reset_index()
edges = edges[['edge','mean popularity','count','mean followers']]

In [ ]:
edges.to_csv('edges2010.csv', index = False)

In [ ]:
nodes = [{'node': j, 'population': i['popularity'],'followers': i['followers']}  for i in gnrs for j in i['genre']]
nodes = pd.DataFrame.from_dict(nodes)
nodes = pd.DataFrame(nodes.groupby(['node']).agg(['mean','count']))
nodes.columns = ['mean popularity','count','mean followers', 'all count']
nodes = nodes.reset_index()
nodes = nodes[['node','mean popularity','count','mean followers']]

In [ ]:
nodes.to_csv('nodes-2010.csv', index = False)